In [1]:
from weavepy import *
from config import BDD_VERSION
sns.set_context("talk")

In [2]:
# --- Parameters for loading data ---
# It is expected that these will be changed rarely
## Variables
climate_variable = "TA"
energy_variable = "SPV"
vars = [climate_variable, energy_variable]
techno = "60" # Techno needs to be specified for energy variables
## Temporal aggregation
aggregation_frequency = "D"
aggregation_function = "mean"
## Selection
country = "FR"
models = []
scenarios = []

In [3]:
data = load_vars([climate_variable, energy_variable], bdd_version = BDD_VERSION,
              countries = [country], technos = ["NA", techno], models = models, scenarios = scenarios, 
              aggregation_frequency = aggregation_frequency, aggregation_function = aggregation_function,
              verbose=True)

Loading country n°1/1 (FR)
|_Variable n°1/2 (TA)
|__Techno n°1/1 (NA)
|__ > File already exists, loading from cache: /Users/bourdin/Library/CloudStorage/OneDrive-Nexus365/WEAVE/PECD4.2/TA_NA_D-mean_FR_PECD4.2.nc
|_Variable n°2/2 (SPV)
|__Techno n°1/1 (60)
|__ > File already exists, loading from cache: /Users/bourdin/Library/CloudStorage/OneDrive-Nexus365/WEAVE/PECD4.2/SPV_60_D-mean_FR_PECD4.2.nc


In [10]:
# --- Period ---
# These can be changed and adjusted easily
historical_period = (1990, 2020)
future_period = (2070, 2100)

In [11]:
# --- Thresholds ---
# These can be changed and adjusted easily
comparisons = { # For each variable, choose among '<', '<=', '>', or '>='
    climate_variable:'<=', 
    energy_variable:'<',
}
thresholds = { # Number in units of variable
    climate_variable: 0,
    energy_variable: 0.05
}

In [12]:
# Identify problematic days
days = {}
for var in vars:
    days[var] = identify_pb_days(data[var], comparisons[var], thresholds[var])
days["compound"] = xr.concat(list(days.values()), dim = "var").all(dim = "var").rename("compound")
# Identify events
events = {}
for var in vars + ["compound"]:
    events[var] = identify_events_whole_base(days[var])

100%|██████████████████████████████████████████| 35/35 [00:00<00:00, 360.02it/s]


## Number of events

In [13]:
#event_count_barplot_multi(list(events.values()), historical_period, future_period)

In [14]:
#nb_event_timeseries_multi(list(events.values()))

## Duration 

In [15]:
event_duration_hist_multi(list(events.values()), historical_period, future_period)

/Users/bourdin/Library/CloudStorage/OneDrive-Nexus365/WEAVE/weave/weavepy/plot_hists.py:84: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/bourdin/Library/CloudStorage/OneDrive-Nexus365/WEAVE/weave/weavepy/plot_hists.py:84: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/bourdin/Library/CloudStorage/OneDrive-Nexus365/WEAVE/weave/weavepy/plot_hists.py:84: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

FigureWidget({
    'data': [{'fill': 'tozeroy',
              'fillcolor': 'rgba(23,60,102,0.15)',
              'hovertemplate': '%{x}<br>proportion: %{y:.3f}<extra></extra>',
              'legendgroup': 'SP126',
              'line': {'color': 'rgb(23,60,102)', 'width': 3},
              'mode': 'lines',
              'name': 'SP126',
              'showlegend': True,
              'type': 'scatter',
              'uid': '98e3a8bc-f376-4afa-9f5c-51b2854558fc',
              'x': {'bdata': ('AAAAAAAA4D8AAAAAAAD4PwAAAAAAAP' ... 'AAAAAAIUAAAAAAAAAhQAAAAAAAACNA'),
                    'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': ('ntXOame10z+e1c5qZ7XTP4mpxFRiKt' ... 'hKTCWmkj/0EHoIPYSOP/QQegg9hI4/'),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'legendgroup': 'SP126',
              'line': {'color': 'rgb(23,60,102)', 'dash': 'dash', 'width': 2},
              'mode': 'lines',
              'name': 'SP126 mean',
              '

## Seasonality

In [19]:
event_seasonality_kde_multi(list(days.values()), historical_period, future_period)

FigureWidget({
    'data': [{'customdata': {'bdata': ('AAAAAADQZsAUqPUrBVpmwClQ61cK5G' ... 'tXCuRlQBWo9SsFWmZAAAAAAADQZkA='),
                             'dtype': 'f8'},
              'hovertemplate': 'DOY: %{customdata:.0f}<br>Density: %{r:.3f}<extra></extra>',
              'legendgroup': np.str_('SP126'),
              'line': {'color': 'rgb(23,60,102)'},
              'mode': 'lines',
              'name': np.str_('SP126'),
              'r': {'bdata': ('Q72Vm2ncxD8evZWbadzEP069lZtp3M' ... 'WbadzEPy69lZtp3MQ/S72Vm2ncxD8='),
                    'dtype': 'f8'},
              'showlegend': True,
              'subplot': 'polar',
              'theta': {'bdata': ('AAAAAACAZsCLLrroogtmwBdddNFFl2' ... 'TRRZdlQIwuuuiiC2ZAAAAAAACAZkA='),
                        'dtype': 'f8'},
              'type': 'scatterpolar',
              'uid': 'fe86959d-2853-448f-b557-3d00ed1de76b'},
             {'customdata': {'bdata': ('AAAAAADQZsAUqPUrBVpmwClQ61cK5G' ... 'tXCuRlQBWo9SsFWmZAAAAAAADQZkA='),
    